In [13]:
from DNAflexpy.core import DNAflexpy
from DNAflexpy.utils import load_feature_data, read_fasta

In [14]:
seqs = read_fasta('../DNAflexpy/data/test_fasta.fa')
seqs

<generator object read_fasta at 0x7f016ab15140>

In [15]:
fdata = load_feature_data('../DNAflexpy/data/lookupNEW.yaml')
fdata.keys()

dict_keys(['NPP', 'DNaseI', 'bendabilityDNase', 'bendabilityConcensus', 'wedge', 'prop', 'freeen', 'gc', 'twistDisp', 'stifness', 'bendingStiffness'])

In [16]:
profile = []

for id, seq in (seqs):
    
    f = DNAflexpy(seqid = id,
                  record = seq,
                  kmer_len = 3,
                  window_size=0,
                  feature='DNaseI',
                  feature_lookup=fdata)

#     print(f)
    profile.append(f)

In [17]:
import pandas as pd
pd.DataFrame(profile)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,sequence1,0.134,0.076,-0.077,-0.033,0.025,0.025,-0.033,-0.033,0.025,...,0.017,-0.077,-0.033,0.025,0.090,0.017,0.017,0.09,0.090,-0.183
1,sequence2,-0.033,0.025,0.090,0.017,0.017,0.090,0.090,-0.183,0.025,...,-0.003,-0.033,0.025,0.025,-0.033,-0.033,0.025,0.09,0.017,0.017
